In [27]:
import pandas
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

wf = pandas.read_csv("/home/yash/Downloads/presidentialspeeches.csv")

In [28]:
##getting main body paragraphs from certain news articles

from newspaper import Article

url = "https://www.gov.uk/government/speeches/pm-climate-ambition-summit-opening-remarks-12-december-2020"
article = Article(url)
article.download()
article.parse()

print(article.text)




Thank you very much Zeinab, thank you Secretary-General Antonio, thank you to my fellow leaders, excellences.

Good afternoon from London, where we are coming to the end of an extraordinary and difficult year, I think with a sudden surge of scientific optimism.

Because after barely 12 months of the pandemic, we’re seeing the vaccine going into the arms of the elderly and vulnerable, vaccines that have been products each and every one of them of vast international efforts in laboratories around the world.

And so my message to you all, is that together we can use scientific advances to protect our entire planet, our biosphere against a challenge far worse, far more destructive even than coronavirus.

By the promethean power of our invention we can begin to defend the earth against the disaster of global warming.

And by that I mean that together we can reduce our emissions, we can radically cut our dependence on fossil fuels, we can change our agricultural practices, and in short we ca

In [29]:
wf = wf.head(500)
wf

,President,Party,from,until,Vice President,title,date,info,speech
0,Donald Trump,Republican,2017,2021,1.0,"January 8, 2020: Statement on Iran",1/8/2020,After the killing of General Qasem Soleimani o...,As long as I am President of the United States...
1,Donald Trump,Republican,2017,2021,1.0,"January 3, 2020: Remarks on the Killing of Qas...",1/3/2020,President Trump announces that the US military...,"Hello, everybody. Well, thank you very much. ..."
2,Donald Trump,Republican,2017,2021,1.0,"October 27, 2019: Statement on the the Death o...",10/27/2019,President Donald Trump announces the death of ...,"Last night, the United States brought the worl..."
3,Donald Trump,Republican,2017,2021,1.0,"September 25, 2019: Press Conference",9/25/2019,President Donald Trump holds a press conferenc...,NaN
4,Donald Trump,Republican,2017,2021,1.0,"September 24, 2019: Remarks at the United Nati...",9/24/2019,President Donald Trump speaks to the 74th sess...,PRESIDENT TRUMP: Thank you very much. Mr. ...
...,...,...,...,...,...,...,...,...,...
495,Herbert Hoover,Republican,1929,1933,1.0,"December 2, 1930: Second State of the Union ad...",12/2/1930,"In his second State of the Union address, Pres...",To the Senate and House of Representatives:I h...
496,Herbert Hoover,Republican,1929,1933,1.0,"October 2, 1930: Address to the American Banke...",10/2/1930,No info available,Members of the American Bankers Association an...
497,Herbert Hoover,Republican,1929,1933,1.0,"July 7, 1930: Message Regarding London Naval T...",7/7/1930,President Hoover delivers a special session me...,To the Senate:In requesting the Senate to conv...
498,Herbert Hoover,Republican,1929,1933,1.0,"June 16, 1930: Message regarding the Smoot-Haw...",6/16/1930,The Smoot-Hawley Tariff raises duties prohibit...,I SHALL approve the tariff bill. This legisla...


In [30]:
wf['Party'] = wf['Party'].replace('Republican', 0)
wf['Party'] = wf['Party'].replace('Democratic', 1)

/tmp/ipykernel_174512/3611045021.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  wf['Party'] = wf['Party'].replace('Democratic', 1)


In [31]:
wf

,President,Party,from,until,Vice President,title,date,info,speech
0,Donald Trump,0,2017,2021,1.0,"January 8, 2020: Statement on Iran",1/8/2020,After the killing of General Qasem Soleimani o...,As long as I am President of the United States...
1,Donald Trump,0,2017,2021,1.0,"January 3, 2020: Remarks on the Killing of Qas...",1/3/2020,President Trump announces that the US military...,"Hello, everybody. Well, thank you very much. ..."
2,Donald Trump,0,2017,2021,1.0,"October 27, 2019: Statement on the the Death o...",10/27/2019,President Donald Trump announces the death of ...,"Last night, the United States brought the worl..."
3,Donald Trump,0,2017,2021,1.0,"September 25, 2019: Press Conference",9/25/2019,President Donald Trump holds a press conferenc...,NaN
4,Donald Trump,0,2017,2021,1.0,"September 24, 2019: Remarks at the United Nati...",9/24/2019,President Donald Trump speaks to the 74th sess...,PRESIDENT TRUMP: Thank you very much. Mr. ...
...,...,...,...,...,...,...,...,...,...
495,Herbert Hoover,0,1929,1933,1.0,"December 2, 1930: Second State of the Union ad...",12/2/1930,"In his second State of the Union address, Pres...",To the Senate and House of Representatives:I h...
496,Herbert Hoover,0,1929,1933,1.0,"October 2, 1930: Address to the American Banke...",10/2/1930,No info available,Members of the American Bankers Association an...
497,Herbert Hoover,0,1929,1933,1.0,"July 7, 1930: Message Regarding London Naval T...",7/7/1930,President Hoover delivers a special session me...,To the Senate:In requesting the Senate to conv...
498,Herbert Hoover,0,1929,1933,1.0,"June 16, 1930: Message regarding the Smoot-Haw...",6/16/1930,The Smoot-Hawley Tariff raises duties prohibit...,I SHALL approve the tariff bill. This legisla...


In [32]:
def remove_punctuation(speech):
    translator = str.maketrans("", "", string.punctuation)
    return text.translate(translator)

In [33]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    #removes stop words
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)


[nltk_data] Downloading package stopwords to /home/yash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
wf.iloc[0]['speech']

'As long as I am President of the United States, Iran will never be allowed to have a nuclear weapon. Good morning. \xa0I’m pleased to inform you: The American people should be extremely grateful and happy no Americans were harmed in last night’s attack by the Iranian regime. \xa0We suffered no casualties, all of our soldiers are safe, and only minimal damage was sustained at our military bases. Our great American forces are prepared for anything. \xa0Iran appears to be standing down, which is a good thing for all parties concerned and a very good thing for the world. No American or Iraqi lives were lost because of the precautions taken, the dispersal of forces, and an early warning system that worked very well. \xa0I salute the incredible skill and courage of America’s men and women in uniform. For far too long — all the way back to 1979, to be exact — nations have tolerated Iran’s destructive and destabilizing behavior in the Middle East and beyond. \xa0Those days are over. \xa0Iran 

In [35]:
for i in range(len(wf) - 1, -1, -1):
        if not (isinstance(wf.iloc[i]['speech'], str)):
                wf = wf.drop(i)

In [36]:
wf

,President,Party,from,until,Vice President,title,date,info,speech
0,Donald Trump,0,2017,2021,1.0,"January 8, 2020: Statement on Iran",1/8/2020,After the killing of General Qasem Soleimani o...,As long as I am President of the United States...
1,Donald Trump,0,2017,2021,1.0,"January 3, 2020: Remarks on the Killing of Qas...",1/3/2020,President Trump announces that the US military...,"Hello, everybody. Well, thank you very much. ..."
2,Donald Trump,0,2017,2021,1.0,"October 27, 2019: Statement on the the Death o...",10/27/2019,President Donald Trump announces the death of ...,"Last night, the United States brought the worl..."
4,Donald Trump,0,2017,2021,1.0,"September 24, 2019: Remarks at the United Nati...",9/24/2019,President Donald Trump speaks to the 74th sess...,PRESIDENT TRUMP: Thank you very much. Mr. ...
6,Donald Trump,0,2017,2021,1.0,"February 5, 2019: State of the Union Address",2/5/2019,"In his second State of the Union Address, Pres...","Madam Speaker, Mr. Vice President, Members of..."
...,...,...,...,...,...,...,...,...,...
495,Herbert Hoover,0,1929,1933,1.0,"December 2, 1930: Second State of the Union ad...",12/2/1930,"In his second State of the Union address, Pres...",To the Senate and House of Representatives:I h...
496,Herbert Hoover,0,1929,1933,1.0,"October 2, 1930: Address to the American Banke...",10/2/1930,No info available,Members of the American Bankers Association an...
497,Herbert Hoover,0,1929,1933,1.0,"July 7, 1930: Message Regarding London Naval T...",7/7/1930,President Hoover delivers a special session me...,To the Senate:In requesting the Senate to conv...
498,Herbert Hoover,0,1929,1933,1.0,"June 16, 1930: Message regarding the Smoot-Haw...",6/16/1930,The Smoot-Hawley Tariff raises duties prohibit...,I SHALL approve the tariff bill. This legisla...


In [37]:
from collections import Counter

def count_unique_word(wf_training):
    count = Counter()
    for i in range(len(wf_training)):
        for word in wf_training.iloc[i]['speech'].split():
            count[word] += 1
    return count

counter = count_unique_word(wf)

In [38]:
print(len(counter))

60017


In [39]:
counter

Counter({'the': 81016,
         'of': 52323,
         'to': 47636,
         'and': 44859,
         'in': 28477,
         'a': 25311,
         'that': 23883,
         'I': 16933,
         'our': 16259,
         'we': 15801,
         'is': 15669,
         'for': 13963,
         'have': 12654,
         'will': 10274,
         'this': 10113,
         'are': 9867,
         'be': 9680,
         'with': 8050,
         'on': 7948,
         'not': 7842,
         'as': 7556,
         'it': 7033,
         'We': 6459,
         'by': 6400,
         'you': 6259,
         'all': 6116,
         'their': 5795,
         'And': 5764,
         'has': 5478,
         'The': 5425,
         'from': 4936,
         'or': 4853,
         'can': 4726,
         'who': 4647,
         'they': 4500,
         'people': 4390,
         'was': 4299,
         'at': 4166,
         'more': 4114,
         'which': 3982,
         'been': 3955,
         'an': 3944,
         'must': 3735,
         'would': 3581,
         'do': 3

In [40]:
num_unique_words = len(counter)

In [41]:
wf_testing = wf[::2]
wf_training = wf[1::2]

In [42]:
wf_testing.reset_index(inplace=True)
wf_testing.drop('index', axis=1, inplace=True)
wf_testing

wf_training.reset_index(inplace=True)
wf_training.drop('index', axis=1, inplace=True)
wf_training

/tmp/ipykernel_174512/375450630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_testing.drop('index', axis=1, inplace=True)
/tmp/ipykernel_174512/375450630.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wf_training.drop('index', axis=1, inplace=True)


,President,Party,from,until,Vice President,title,date,info,speech
0,Donald Trump,0,2017,2021,1.0,"January 3, 2020: Remarks on the Killing of Qas...",1/3/2020,President Trump announces that the US military...,"Hello, everybody. Well, thank you very much. ..."
1,Donald Trump,0,2017,2021,1.0,"September 24, 2019: Remarks at the United Nati...",9/24/2019,President Donald Trump speaks to the 74th sess...,PRESIDENT TRUMP: Thank you very much. Mr. ...
2,Donald Trump,0,2017,2021,1.0,"January 19, 2019: Remarks about the US Souther...",1/19/2019,President Donald Trump speaks about what he se...,"THE PRESIDENT: Just a short time ago, I had th..."
3,Donald Trump,0,2017,2021,1.0,"March 19, 2018: Remarks on Combating the Opioi...",3/19/2018,President Trump speaks in New Hampshire about ...,"THE PRESIDENT: Thank you to our First Lady, Me..."
4,Donald Trump,0,2017,2021,1.0,"February 1, 2018: Remarks at the House and Sen...",2/1/2018,President Trump addresses the Republican membe...,"THE PRESIDENT: Thank you, Paul and Mitch, for ..."
...,...,...,...,...,...,...,...,...,...
238,Herbert Hoover,0,1929,1933,1.0,"September 22, 1931: Message on the Gold Standard",9/22/1931,President Hoover addresses Britain’s departure...,GREAT BRITAIN'S DEPARTURE FROM THE GOLD STANDA...
239,Herbert Hoover,0,1929,1933,1.0,"February 26, 1931: Veto Messages Regarding Eme...",2/26/1931,The Emergency Adjusted Compensation Act allow...,To the House of Representatives:I return herew...
240,Herbert Hoover,0,1929,1933,1.0,"December 9, 1930: Message Regarding Unemployme...",12/9/1930,President Hoover addresses legislation in Cong...,"THE PRESIDENT said:""I observe that measures ha..."
241,Herbert Hoover,0,1929,1933,1.0,"October 2, 1930: Address to the American Banke...",10/2/1930,No info available,Members of the American Bankers Association an...


In [43]:
training_sentences = wf_training.speech.to_numpy()
training_labels = wf_training.Party.to_numpy()
validation_sentences = wf_testing.speech.to_numpy()
validation_labels = wf_testing.Party.to_numpy()

In [44]:
training_sentences.shape, validation_sentences.shape

((243,), (244,))

In [45]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(training_sentences)

In [46]:
word_index = tokenizer.word_index

In [47]:
training_sequences = tokenizer.texts_to_sequences(training_sentences)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)

In [48]:
#padding sequences to have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences

#max number of words per sequence
max_length = 100

training_padded = pad_sequences(training_sequences, maxlen=max_length, padding="post", truncating="post")
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding="post", truncating="post")
training_padded.shape, validation_padded.shape

((243, 100), (244, 100))

In [49]:
reverse_word_index_dictionary = dict([(idx, word) for (word, idx) in word_index.items()])

In [50]:
def decode(sequence):
    #takes the sequence and decodes it according to the reverse dictionary
    return " ".join([reverse_word_index_dictionary.get(idx, "?") for idx in sequence])

In [51]:
#LSTM Model
import tensorflow.keras as keras


model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, 32))

#dropout rate drops out a certain percentage of input data so that the
#model learns more robust ways of learning and does not overfit
model.add(layers.LSTM(64, dropout=0.1))
model.add(layers.Dense(128, activation='sigmoid'))
model.add(layers.Dense(64, activation='sigmoid'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [52]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.001)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [53]:
model.fit(training_padded, training_labels, epochs=7, validation_data=(validation_padded, validation_labels), verbose=2)

Epoch 1/7


2024-10-17 18:30:29.356681: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


8/8 - 6s - 750ms/step - accuracy: 0.4239 - loss: 0.7604 - val_accuracy: 0.5902 - val_loss: 0.6783
Epoch 2/7
8/8 - 0s - 43ms/step - accuracy: 0.5802 - loss: 0.6858 - val_accuracy: 0.5902 - val_loss: 0.6913
Epoch 3/7
8/8 - 0s - 43ms/step - accuracy: 0.5802 - loss: 0.6946 - val_accuracy: 0.5902 - val_loss: 0.6773
Epoch 4/7
8/8 - 0s - 37ms/step - accuracy: 0.5802 - loss: 0.6802 - val_accuracy: 0.5902 - val_loss: 0.6769
Epoch 5/7
8/8 - 0s - 39ms/step - accuracy: 0.5802 - loss: 0.6809 - val_accuracy: 0.5902 - val_loss: 0.6790
Epoch 6/7
8/8 - 0s - 41ms/step - accuracy: 0.5802 - loss: 0.6656 - val_accuracy: 0.5902 - val_loss: 0.6718
Epoch 7/7
8/8 - 0s - 41ms/step - accuracy: 0.6296 - loss: 0.6245 - val_accuracy: 0.5861 - val_loss: 0.6562
